In [2]:
from pykrx import stock
import pandas as pd
import re
import numpy as np

In [3]:
df = pd.read_excel('삼성전자2025.xlsx')

In [4]:
def normalize(text):
    # 특수문자 제외, 공백에 대한 처리
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 금융 문장 감성 분석 모델
MODEL = "snunlp/KR-FinBert-SC"

# 1) 토크나이저/모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 2) 파이프라인 생성
nlp = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

# 테스트
sentence = "삼성전자는 반도체 업황 개선으로 실적이 대폭 증가했다."
result = nlp(sentence)

print(result)

Device set to use cpu


[[{'label': 'negative', 'score': 5.399243309511803e-05}, {'label': 'neutral', 'score': 6.116756412666291e-05}, {'label': 'positive', 'score': 0.999884843826294}]]


c:\Users\rkdtk\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
df['제목'] = df['제목'].map(normalize)

In [7]:
df = df[['일자', '제목']]
df.rename(columns={'일자':'date', '제목':'title'}, inplace=True)

In [8]:
df

,date,title
0,20251203,2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE
1,20251203,삼성전자 GDDR7 D램으로 대통령상 수상
2,20251203,삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개
3,20251203,포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE
4,20251203,삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만
...,...,...
16239,20250101,삼성전자 레인보우로보틱스 품었다 미래 로봇 개발 가속도
16240,20250101,로봇기업 품은 삼성전자 대표가 직접 지휘봉 잡는다
16241,20250101,빅테크 새 격전지는 로봇 삼성전자도 휴머노이드 만든다
16242,20250101,삼성전자 AI RAN기술 시연 통신 AI 융합기술로 미래 준비한다


In [9]:
df2 = df.copy()

In [10]:
df2 = df[:100]

In [11]:
df2

,date,title
0,20251203,2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE
1,20251203,삼성전자 GDDR7 D램으로 대통령상 수상
2,20251203,삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개
3,20251203,포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE
4,20251203,삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만
...,...,...
95,20251202,삼성전자 첫 3단 폴더블폰 갤럭시 Z 트라이폴드 데뷔 힌지 대화면 완성도 개선
96,20251202,주머니 속 태블릿 현실로 삼성전자 Z 트라이폴드 로 초격차 승부
97,20251202,삼성전자 대화면 갤럭시 Z 트라이폴드 공개 출고가 359만원
98,20251202,삼성전자 첫 3단 폴더블 갤럭시 Z 트라이폴드 공개 359만 원


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
from tqdm import tqdm

# 1) 모델 로드
MODEL = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

nlp = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)


# 2) 제목 감성 점수 산출 함수
def get_sentiment(title: str) -> float:
    if not isinstance(title, str):
        return 0.0
    scores = nlp(title)[0]  # [{label, score}, {label, score}, ...]
    pos = [s['score'] for s in scores if s['label'] == 'positive'][0]
    neg = [s['score'] for s in scores if s['label'] == 'negative'][0]
    return pos - neg


# 4) 감성 점수 생성
tqdm.pandas(desc="Scoring titles")
df2["sentiment"] = df2["title"].progress_apply(get_sentiment)

# 5) 종목/일자별 평균 팩터 생성
# factor = (
#     df.groupby(["date", "ticker"])["sentiment"]
#     .mean()
#     .reset_index()
# )

# factor.to_csv("sentiment_factor.csv", index=False)
# print("저장 완료 → sentiment_factor.csv")

df

Device set to use cpu
c:\Users\rkdtk\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Scoring titles: 100%|██████████| 100/100 [00:03<00:00, 30.29it/s]
C:\Users\rkdtk\AppData\Local\Temp\ipykernel_33568\1761116401.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["sentiment"] = df2["title"].progress_apply(get_sentiment)


,date,title
0,20251203,2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE
1,20251203,삼성전자 GDDR7 D램으로 대통령상 수상
2,20251203,삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개
3,20251203,포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE
4,20251203,삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만
...,...,...
16239,20250101,삼성전자 레인보우로보틱스 품었다 미래 로봇 개발 가속도
16240,20250101,로봇기업 품은 삼성전자 대표가 직접 지휘봉 잡는다
16241,20250101,빅테크 새 격전지는 로봇 삼성전자도 휴머노이드 만든다
16242,20250101,삼성전자 AI RAN기술 시연 통신 AI 융합기술로 미래 준비한다


In [13]:
df2

,date,title,sentiment
0,20251203,2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE,0.999440
1,20251203,삼성전자 GDDR7 D램으로 대통령상 수상,0.999800
2,20251203,삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개,0.999762
3,20251203,포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE,0.044790
4,20251203,삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만,0.999831
...,...,...,...
95,20251202,삼성전자 첫 3단 폴더블폰 갤럭시 Z 트라이폴드 데뷔 힌지 대화면 완성도 개선,0.999823
96,20251202,주머니 속 태블릿 현실로 삼성전자 Z 트라이폴드 로 초격차 승부,0.004698
97,20251202,삼성전자 대화면 갤럭시 Z 트라이폴드 공개 출고가 359만원,0.999694
98,20251202,삼성전자 첫 3단 폴더블 갤럭시 Z 트라이폴드 공개 359만 원,0.999825


In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 금융 문장 감성 분석 모델
MODEL = "snunlp/KR-FinBert-SC"

# 1) 토크나이저/모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 2) 파이프라인 생성
nlp = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
    # return_all_scores=True
)

# 테스트
sentence = "삼성전자는 반도체 업황 개선으로 실적이 대폭 증가했다."
result = nlp(sentence)

print(result)

Device set to use cpu


[{'label': 'positive', 'score': 0.999884843826294}]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# 금융 문장 감성 분석 모델
MODEL = "snunlp/KR-FinBert-SC"

# 1) 토크나이저/모델 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# 2) 파이프라인 생성
nlp = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
    # return_all_scores=True
)

result = []
# 테스트
sentence = df['title']
for title in df['title']:
    result.append(nlp(title))
    if len(result) % 1000 == 0:
        print(f"Processed {len(result)} titles")


result

Device set to use cpu


Processed 1000 titles
Processed 2000 titles
Processed 3000 titles
Processed 4000 titles
Processed 5000 titles
Processed 6000 titles
Processed 7000 titles
Processed 8000 titles
Processed 9000 titles
Processed 10000 titles
Processed 11000 titles
Processed 12000 titles
Processed 13000 titles
Processed 14000 titles
Processed 15000 titles
Processed 16000 titles


[[{'label': 'positive', 'score': 0.9995007514953613}],
 [{'label': 'positive', 'score': 0.9998449087142944}],
 [{'label': 'positive', 'score': 0.999800980091095}],
 [{'label': 'neutral', 'score': 0.9485037326812744}],
 [{'label': 'positive', 'score': 0.9998810291290283}],
 [{'label': 'neutral', 'score': 0.8645607233047485}],
 [{'label': 'positive', 'score': 0.9997842907905579}],
 [{'label': 'positive', 'score': 0.9996848106384277}],
 [{'label': 'positive', 'score': 0.9998683929443359}],
 [{'label': 'neutral', 'score': 0.9998403787612915}],
 [{'label': 'positive', 'score': 0.9998750686645508}],
 [{'label': 'neutral', 'score': 0.9977598190307617}],
 [{'label': 'positive', 'score': 0.9995802044868469}],
 [{'label': 'neutral', 'score': 0.9875701069831848}],
 [{'label': 'positive', 'score': 0.9998804330825806}],
 [{'label': 'neutral', 'score': 0.9866625666618347}],
 [{'label': 'neutral', 'score': 0.871303915977478}],
 [{'label': 'positive', 'score': 0.9738455414772034}],
 [{'label': 'positi

In [24]:
df['title'].head(20)

0              2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE
1                          삼성전자 GDDR7 D램으로 대통령상 수상
2              삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개
3              포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE
4                  삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만
5                    비즈톡 메모리 가격 상승에 웃을 수만은 없는 삼성전자
6            대통령상만 11번째 삼성전자 세계 최초 D램 기술로 AI 추론 선도
7          삼성전자 GDDR7 대통령표창 받아...AI 연산에 활용되며 날개 다나
8               삼성전자 HBM4 내부 품질 테스트 통과 본격 양산 준비 완료
9              11번가 삼성전자 로보락 등 올해의 브랜드 와 연말 감사제 실시
10               GDDR7 대통령상 수상 삼성전자 AI 메모리 리더십 공고화
11         서울데이터랩 삼성전자 SK 하이닉스 등 개장 직후 인기 검색 종목 20
12               삼성전자 아트 스토어에 아트 바젤 마이애미 비치 컬렉션 공개
13           코스피 개인 순매수 속 4000선 안팎 공방 삼성전자 1 대 오름세
14                   4분기 실적 서프라이즈 전망에 삼성전자 매수세 몰렸다
15                            4000피 회복한 코스피 삼성전자 2
16                 삼성전자 아트 바젤 마이애미 비치 컬렉션 4K로 선보인다
17    리포트 브리핑 삼성전자 HBM4 연내 승인 전망 목표가 160 000원 KB증권
18            삼성전자 삼성 아트 스토어에 아트 바젤 마이애미 비치 컬렉션 공개
19     와이씨 내년 삼성전자 SK하이닉스 수요 확대

In [26]:
df['title'].tail(4)

16240             로봇기업 품은 삼성전자 대표가 직접 지휘봉 잡는다
16241           빅테크 새 격전지는 로봇 삼성전자도 휴머노이드 만든다
16242    삼성전자 AI RAN기술 시연 통신 AI 융합기술로 미래 준비한다
16243    삼성전자 레인보우로보틱스 최대주주 지위 확보 미래로봇 개발 가속화
Name: title, dtype: object

In [28]:
df['ticker'] = '005930'

In [29]:
df

,date,title,ticker
0,20251203,2025년 TTA 시험인증 대상 특별상 삼성전자 갤럭시S25FE,005930
1,20251203,삼성전자 GDDR7 D램으로 대통령상 수상,005930
2,20251203,삼성전자 삼성 아트 스토어 아트 바젤 마이애미 비치 컬렉션 공개,005930
3,20251203,포토 TTA 시험인증 대상 특별상에 삼성전자 갤럭시 S25 FE,005930
4,20251203,삼성전자 베트남 국적 임원 첫 탄생 현지 진출 17년 만,005930
...,...,...,...
16239,20250101,삼성전자 레인보우로보틱스 품었다 미래 로봇 개발 가속도,005930
16240,20250101,로봇기업 품은 삼성전자 대표가 직접 지휘봉 잡는다,005930
16241,20250101,빅테크 새 격전지는 로봇 삼성전자도 휴머노이드 만든다,005930
16242,20250101,삼성전자 AI RAN기술 시연 통신 AI 융합기술로 미래 준비한다,005930


In [31]:
df2['ticker'] = '005930'

C:\Users\rkdtk\AppData\Local\Temp\ipykernel_33568\2809714094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ticker'] = '005930'


In [34]:
df2.drop(columns='sentiment', inplace = True)

C:\Users\rkdtk\AppData\Local\Temp\ipykernel_33568\833820993.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns='sentiment', inplace = True)


In [38]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import pandas as pd
from tqdm import tqdm



# 1) FinBERT 모델 로드

MODEL = "snunlp/KR-FinBert-SC"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

nlp = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)



# 2) 감성 점수 추출 함수
#    Sentiment = P(positive) - P(negative)

def get_sentiment(text: str) -> float:
    if not isinstance(text, str):
        return 0.0
    
    scores = nlp(text)[0]   # [{'label': 'positive', 'score': ...}, ...]
    
    pos = [s['score'] for s in scores if s['label'] == 'positive'][0]
    neg = [s['score'] for s in scores if s['label'] == 'negative'][0]
    
    return pos - neg        # 최종 감성 점수


# ------------------------------------------------------
# 3) 뉴스 데이터 예시 구조
#    df 컬럼: date(YYYY-MM-DD), ticker, title
# ------------------------------------------------------
# df = pd.read_csv("news.csv")  # 예시
# df["date"] = pd.to_datetime(df["date"])


# ------------------------------------------------------
# 4) 감성 점수 계산
# ------------------------------------------------------
tqdm.pandas(desc="Calculating Sentiment")
df["sentiment"] = df["title"].progress_apply(get_sentiment)


# ------------------------------------------------------
# 5) 종목/일자별 1일 감성 만들기 (여러 뉴스 평균)
# ------------------------------------------------------
daily_sentiment = (
    df.groupby(["ticker", "date"])["sentiment"]
      .mean()
      .reset_index()
      .sort_values(["ticker", "date"])
)


# ------------------------------------------------------
# 6) 종목별 3일 롤링 평균 감성 팩터 생성 (핵심)
# ------------------------------------------------------
daily_sentiment["sentiment_3d"] = (
    daily_sentiment
    .groupby("ticker")["sentiment"]
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)



Device set to use cpu
c:\Users\rkdtk\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Calculating Sentiment: 100%|██████████| 16244/16244 [08:15<00:00, 32.81it/s]


In [39]:
daily_sentiment

,ticker,date,sentiment,sentiment_3d
0,005930,20250101,0.523322,0.523322
1,005930,20250102,0.393929,0.458625
2,005930,20250103,0.646831,0.521361
3,005930,20250104,0.124300,0.388354
4,005930,20250105,0.140927,0.304020
...,...,...,...,...
332,005930,20251129,0.499914,0.481177
333,005930,20251130,0.720633,0.507541
334,005930,20251201,0.366668,0.529072
335,005930,20251202,0.672841,0.586714
